# **Data Preprocess**

## Importation & Data loading

Preparing necessary packages (may need to add more):

In [ ]:
import numpy as np  # Work with multi-dimenional data
import pandas as pd # Work with relational data
import matplotlib.pyplot as plt # Visualize data
import seaborn as sns # Visualize data base on matplotlib
import nltk #
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from scipy import sparse

nltk.download('punkt')

import warnings
warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split

%matplotlib inline

Load data from files

In [ ]:
directory = 'data/dataset/raw/'

# This method use to extract all the file in the input list
def extract_data(files):
  data_x_raw = list()
  data_y_raw = list()
  for file in files:
    with open(directory + file) as f:
      for line in f:
        line = line.strip('\n')
        x_raw, y_raw = line.split(sep=';')

        data_x_raw.append(x_raw)
        data_y_raw.append(y_raw)

  return data_x_raw, data_y_raw

In [ ]:
# Get all data

X_raw, y_raw = extract_data(['train.txt', 'val.txt', 'test.txt'])
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X_raw, y_raw, test_size = 0.2)

print("Traning data's size is:", len(X_train_raw))
print("Test data's size is:", len(X_test_raw))

## Inital feature Extraction


Removing the stopword

In [ ]:
file = open("data/stopwords/stop_words_english.txt", 'r', encoding='utf-8')
stopword_list = file.read().split('\n')
file.close()

Vectorize the data:

1.   Bag Of Words model (https://en.wikipedia.org/wiki/Bag-of-words_model)
2.   BoW/TF-IDF model (https://en.wikipedia.org/wiki/Tf%E2%80%93idf)



In [ ]:
# Scikit CountVectorizer:
# Documentation: (https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

count_vector = CountVectorizer(stop_words=stopword_list, ngram_range=(1, 2))
count_vector.fit(X_train_raw, X_test_raw)
X_train_bow = count_vector.transform(X_train_raw)
X_test_bow = count_vector.transform(X_test_raw)
dictionary = count_vector.get_feature_names_out()

print("Shape of preprocessed training data X using BoW model is: ", X_train_bow.shape)

In [ ]:
# Scikit TF-IDF:
# Documentation: (https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer)

tfidf_vector = TfidfVectorizer(stop_words=stopword_list, ngram_range=(1, 2))
tfidf_vector.fit(X_train_raw, X_test_raw)
X_train_tfidf = tfidf_vector.transform(X_train_raw)
X_test_tfidf = tfidf_vector.transform(X_test_raw)

print("Shape of preprocessed training data X using BoW/TF-IDF model is: ", X_train_tfidf.shape)

Cast output list to ndarray

In [ ]:
y_train = np.asarray(y_train_raw)
y_test = np.asarray(y_test_raw)

*Luckily we're quite done with the abbreviations!*

Another way: Using Word2Vec to capture the semantics
(Um, we'd talk about that later, ok?)


## Initial exploratory Data Analysis (EDA)

Average length of the comment:

In [ ]:
# Calculate average number of characters
sum_of_chars = sum([len(x) for x in X_train_raw])
mean_number_chars = sum_of_chars / len(X_train_raw)

# Calculate average number of words
sum_of_words = sum([len(x.split()) for x in X_train_raw])
mean_number_words = sum_of_words / len(X_train_raw)

# Print
print('The mean number of characters of each line is: %d' % mean_number_chars)
print('The mean number of words of each line is: %d' % mean_number_words)

Statistic of output labels:

In [ ]:
# Number of labels:
labels = np.unique(y_train)
print('Number of labels is: %d' % len(labels))
print('Labels: ' + ', '.join(labels))
print('\n----------------------------------------------------------------------------\n')

# Plot:
df = pd.DataFrame(
  dict(
    labels = labels,
    label_elements_count = [y_train_raw.count(label) for label in labels]
  )
)
df = df.sort_values('label_elements_count')

plt.title(label='Labels statistics')
plt.xlabel('Labels')
plt.ylabel('Count')

plt.bar('labels', 'label_elements_count', data=df)

plt.show()

Statistics of words with the highest frequency of each label:

In [ ]:
def show_highest_frequency_words():
  vect_arr = X_train_bow.toarray()
  vect_aggr = np.zeros(shape=(len(labels), vect_arr.shape[1]))

  for idx in range(len(vect_arr)):
    label_idx = int(np.where(labels==y_train[idx])[0][0])

    vect_aggr[label_idx] += vect_arr[idx]

  fig, axs = plt.subplots(3, 2)

  for label_idx in range(len(labels)):
    axs_x, axs_y = label_idx // 2, label_idx % 2

    plt_ref = axs[axs_x][axs_y]
    plt_ref.set_title(labels[label_idx].upper())

    x_axis, y_axis = list(), list()
    n_argmax = np.argpartition(vect_aggr[label_idx], -7)[-7:]
    n_argmax = n_argmax[np.argsort(vect_aggr[label_idx][n_argmax])]
    for i in n_argmax:
      x_axis.append(dictionary[i])
      y_axis.append(vect_aggr[label_idx][i])

    x_axis = np.asarray(x_axis)
    y_axis = np.asarray(y_axis)

    plt_ref.barh(x_axis, y_axis)

  # Adjust layout
  plt.tight_layout()

  # Show the plot
  plt.show()

show_highest_frequency_words()

## Data Fixing

This part will eliminate all the words that have bad influence to our data.
All the words will be stored in a file stored in file *filtered_words.txt*

* Words appears so many times but don't contribute much (*feel*, *feeling*,...).
* Words have no mean (*aa*,*ab*,...)
* Words appears only a few

In [ ]:
file = open("data/stopwords/filtered_words.txt", 'r', encoding='utf-8')
filtered_words = file.read().split('\n')

file = open("data/stopwords/stop_words_english.txt", 'r', encoding='utf-8')
stop_words = file.read().split('\n')

new_stopwords = stopword_list + stop_words + filtered_words
unfiltered_stopwords = stop_words + stopword_list

print(len(new_stopwords), len(unfiltered_stopwords))

Create tokenize function with the help of nltk packages

In [ ]:
stemmer = PorterStemmer()

def tokenize(text):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in new_stopwords]
    stems = [stemmer.stem(token) for token in tokens]
    return stems

def unfiltered_tokenize(text):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in unfiltered_stopwords]
    stems = [stemmer.stem(token) for token in tokens]
    return stems

Fit the data again

In [ ]:
# Dictionary semi_filtered: Using basic stopwords only

count_vector = CountVectorizer(stop_words=unfiltered_stopwords, tokenizer=unfiltered_tokenize, ngram_range=(1, 2))
count_vector.fit(X_train_raw, X_test_raw)
X_train_bow_semi_filtered = count_vector.transform(X_train_raw)
X_test_bow_semi_filtered = count_vector.transform(X_test_raw)
dictionary_semi_filtered = count_vector.get_feature_names_out()

In [ ]:
# Dictionary semi_mono: Using basic filters, containing only 1-gram, used to perform LASSO-filter to obtain L1 dict

count_vector = CountVectorizer(stop_words=unfiltered_stopwords, tokenizer=unfiltered_tokenize, ngram_range=(1, 1))
count_vector.fit(X_train_raw, X_test_raw)
X_train_bow_semi_mono = count_vector.transform(X_train_raw)
X_test_bow_semi_mono = count_vector.transform(X_test_raw)
dictionary_semi_mono = count_vector.get_feature_names_out()

Sorting out the useless stopwords to make L1 dictionary. We perform LASSO Logistic Regression on dictionary_semi_mono to sort out all useless stopwords (criterion: L2 norm of the L1 weight = 0), then make it a new stopword list.


In [ ]:
from sklearn.linear_model import LogisticRegression
linear_semi_mono = LogisticRegression(penalty='l1', solver='saga')
linear_semi_mono.fit(X_train_bow_semi_mono, y_train)

In [ ]:
print(X_train_bow_semi_mono.shape)

In [ ]:
print(linear_semi_mono.coef_.shape)
lin_coef = linear_semi_mono.coef_.T

In [ ]:
def norm2(a: tuple):
    res = 0
    for i in a:
        res += i**2
    return res

In [ ]:
useless_coef = list()
for i in range(lin_coef.shape[0]):
    if norm2(lin_coef[i]) == 0:
        useless_coef.append(i)

In [ ]:
print(len(useless_coef))

In [ ]:
with open("data/stopwords/useless.txt", 'w') as file:
    for i in useless_coef:
        file.write(dictionary_semi_mono[i])
        file.write('\n')

In [ ]:
file = open("data/stopwords/useless.txt", 'r', encoding='utf-8')
useless = file.read().split('\n')

L1_stopwords = unfiltered_stopwords + useless

In [ ]:
def useless_tokenize(text):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in L1_stopwords]
    stems = [stemmer.stem(token) for token in tokens]
    return stems

In [ ]:
# Dictionary L1: Obtained by selecting useful tokens from semi_mono using L1 decomposition.

count_vector = CountVectorizer(stop_words=L1_stopwords, tokenizer=useless_tokenize, ngram_range=(1, 2))
count_vector.fit(X_train_raw, X_test_raw)
X_train_bow_L1 = count_vector.transform(X_train_raw)
X_test_bow_L1 = count_vector.transform(X_test_raw)
dictionary_L1 = count_vector.get_feature_names_out()

In [ ]:
print(len(dictionary_semi_filtered), len(dictionary_L1), len(dictionary_semi_mono))

Fixing results:

In [ ]:
show_highest_frequency_words()

## Export data to files

In [ ]:
save_directory = "data/dataset/processed/"

sparse.save_npz(save_directory + "X_train_bow.npz", X_train_bow)
sparse.save_npz(save_directory + "X_test_bow.npz", X_test_bow)

sparse.save_npz(save_directory + "X_train_tfidf.npz", X_train_tfidf)
sparse.save_npz(save_directory + "X_test_tfidf.npz", X_test_tfidf)

np.savetxt(save_directory + "y_train.txt", y_train, fmt='%s')
np.savetxt(save_directory + "y_test.txt", y_test, fmt='%s')

## Variables summary




---


**X_train_raw**, **y_train_raw**: ***list***

Raw training data which is extracted directly from files.

---


**X_test**, **y_test**: ***list***

  Raw testing data which is extracted directly from files.

---


**stopword_list**: ***list***

 List of stop words used in BoW and BoW/TF-IDF models

---


**dictionary**: ***numpy.ndarray***

 Dictionary of words which is used to vectorized the data and number of occurences.

---


**X_train_bow**: ***scipy.sparse._csr.csr_matrix***

 Training data input which is vectorized by the BoW model.

---


**X_train_tfidf**: ***scipy.sparse._csr.csr_matrix***

 Training data input which is vectorized by the BoW/TFIDF model

---


**X_test_bow**: ***scipy.sparse._csr.csr_matrix***

 Testing data input which is vectorized by the BoW model.

---


**X_test_tfidf**: ***scipy.sparse._csr.csr_matrix***

 Testing data input which is vectorized by the BoW/TFIDF model

---


**y_train**: ***numpy.ndarray***

 Preprocessed traning data output


---